In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import math
import pickle

In [2]:
le = preprocessing.LabelEncoder()

In [3]:
data = pd.read_csv("healthcare-dataset-stroke-data.csv")

In [4]:
data = data.drop(labels="id",axis=1)

In [5]:
data.columns

Index(['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke'],
      dtype='object')

In [6]:
D = data.copy()

In [7]:
D.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [8]:
X = data[["gender","age","hypertension","heart_disease","ever_married","work_type","Residence_type","avg_glucose_level","bmi","smoking_status"]]
X = X.T.values
# to store the gender, ever_married, work_type, Residence_type, smoking_status
invTransform = []

aa = le.fit_transform(X[0])
invTransform.append(le.inverse_transform(aa))
D['gender']=aa

aa = le.fit_transform(X[4])
invTransform.append(le.inverse_transform(aa))
D["ever_married"] = aa

aa = le.fit_transform(X[5])
invTransform.append(le.inverse_transform(aa))
D['work_type'] = aa

aa = le.fit_transform(X[6])
invTransform.append(le.inverse_transform(aa))
D['Residence_type'] = aa

aa = le.fit_transform(X[9])
invTransform.append(le.inverse_transform(aa))
D['smoking_status'] = aa

In [23]:
invTransform

[array(['Male', 'Female', 'Male', ..., 'Female', 'Male', 'Female'],
       dtype=object),
 array(['Yes', 'Yes', 'Yes', ..., 'Yes', 'Yes', 'Yes'], dtype=object),
 array(['Private', 'Self-employed', 'Private', ..., 'Self-employed',
        'Private', 'Govt_job'], dtype=object),
 array(['Urban', 'Rural', 'Rural', ..., 'Rural', 'Rural', 'Urban'],
       dtype=object),
 array(['formerly smoked', 'never smoked', 'never smoked', ...,
        'never smoked', 'formerly smoked', 'Unknown'], dtype=object)]

In [24]:
D.describe()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000
mean,0.414286,43.226614,0.097456,0.054012,0.656164,2.167710,0.508023,106.147677,28.862035,1.376908,0.048728
std,0.493044,22.612647,0.296607,0.226063,0.475034,1.090293,0.499985,45.283560,7.699562,1.071534,0.215320
min,0.000000,0.080000,0.000000,0.000000,0.000000,0.000000,0.000000,55.120000,10.300000,0.000000,0.000000
25%,0.000000,25.000000,0.000000,0.000000,0.000000,2.000000,0.000000,77.245000,23.800000,0.000000,0.000000
50%,0.000000,45.000000,0.000000,0.000000,1.000000,2.000000,1.000000,91.885000,28.100000,2.000000,0.000000
75%,1.000000,61.000000,0.000000,0.000000,1.000000,3.000000,1.000000,114.090000,32.800000,2.000000,0.000000
max,2.000000,82.000000,1.000000,1.000000,1.000000,4.000000,1.000000,271.740000,97.600000,3.000000,1.000000


In [26]:
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [25]:
D.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.6,1,1
1,0,61.0,0,0,1,3,0,202.21,28.1,2,1
2,1,80.0,0,1,1,2,0,105.92,32.5,2,1
3,0,49.0,0,0,1,2,1,171.23,34.4,3,1
4,0,79.0,1,0,1,3,0,174.12,24.0,2,1


In [10]:
D.fillna(value = D['bmi'].median(), inplace = True)

In [11]:
d = D[D['gender'] == 'Other'].index
D.drop(d, inplace = True)

In [12]:
x = D.drop('stroke', axis = 1)
y = D['stroke']

In [13]:
from imblearn.over_sampling import RandomOverSampler

In [14]:
os = RandomOverSampler(sampling_strategy = 1)
x_ros, y_ros = os.fit_resample(x, y)
x = pd.DataFrame(x_ros)
y = pd.DataFrame(y_ros)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.75, shuffle = True, random_state = 1)

In [29]:
knn = KNeighborsClassifier(n_neighbors = 2)
knn.fit(x_train, y_train)
knn_predicted = knn.predict(x_test)
knn_conf_matrix = confusion_matrix(y_test, knn_predicted)
knn_acc_score = accuracy_score(y_test, knn_predicted)

In [30]:
print("Accuracy of K-NeighborsClassifier:",knn_acc_score*100,'\n')
print(classification_report(y_test, knn_predicted))

Accuracy of K-NeighborsClassifier: 97.53187988482107 

              precision    recall  f1-score   support

           0       1.00      0.95      0.97      1200
           1       0.95      1.00      0.98      1231

    accuracy                           0.98      2431
   macro avg       0.98      0.97      0.98      2431
weighted avg       0.98      0.98      0.98      2431



In [31]:
Pkl_Filename = "/Users/rohith/git/stroke_detect/models/HCD_KNN_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(knn, file)

RFC

In [32]:
rfc = RandomForestClassifier(100)
rfc.fit(x_train, y_train)
rfc_predicted = rfc.predict(x_test)
rfc_conf_matrix = confusion_matrix(y_test, rfc_predicted)
rfc_acc_score = accuracy_score(y_test, rfc_predicted)

In [33]:
print("Accuracy of RandomForestClassifier:",rfc_acc_score*100,'\n')
print(classification_report(y_test, rfc_predicted))

Accuracy of RandomForestClassifier: 99.0538872891814 

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1200
           1       0.98      1.00      0.99      1231

    accuracy                           0.99      2431
   macro avg       0.99      0.99      0.99      2431
weighted avg       0.99      0.99      0.99      2431



In [34]:
Pkl_Filename = "/Users/rohith/git/stroke_detect/models/RF_KNN_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(rfc, file)

In [35]:
x_train

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
8457,1,78.0,1,0,0,2,1,113.01,24.0,2
9614,0,67.0,1,0,1,2,0,179.12,28.1,1
2042,1,8.0,0,0,0,4,1,104.51,20.6,0
3474,0,64.0,0,0,1,3,0,65.46,32.5,1
1587,1,2.0,0,0,0,4,1,88.54,17.5,0
...,...,...,...,...,...,...,...,...,...,...
2895,1,46.0,0,0,1,2,1,137.77,29.3,2
7813,0,76.0,0,0,0,2,1,89.96,28.1,0
905,0,31.0,0,0,1,2,0,76.26,35.6,2
5192,1,71.0,0,1,1,2,1,81.76,28.1,3
